In [1]:
import os

os.chdir('C:/Users/EunJin/Desktop/dacon/LG AI')
os.getcwd()

'C:\\Users\\EunJin\\Desktop\\dacon\\LG AI'

In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
PATH = 'C:/Users/EunJin/Desktop/dacon/LG AI/'
train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [5]:
train_err['year'] = train_err['time'].astype(str).apply(lambda x: int(x[:4]))
train_err['month'] = train_err['time'].astype(str).apply(lambda x: int(x[4:6]))
train_err['day'] = train_err['time'].astype(str).apply(lambda x: int(x[6:8]))

train_err['hour'] = train_err['time'].astype(str).apply(lambda x: int(x[8:10]))
train_err['min'] = train_err['time'].astype(str).apply(lambda x: int(x[10:12]))
train_err['sec'] = train_err['time'].astype(str).apply(lambda x: int(x[12:14]))

In [6]:
train_err['date'] = train_err['time'].astype(str).apply(lambda x: int(x[4:8]))

## err_type

In [7]:
pivot = pd.pivot_table(train_err, index = ['user_id'],columns = 'errtype', values = ['time'],aggfunc='count',fill_value=0)
df_errtype = pd.DataFrame(pivot.to_records())

In [8]:
df_errtype = df_errtype.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errtype")) 

## err_day

In [9]:
pivot2 = pd.pivot_table(train_err, index = ['user_id'],columns = 'date', values = ['time'],aggfunc='count',fill_value=0)
df_errday = pd.DataFrame(pivot2.to_records())

In [10]:
df_errday = df_errday.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","day")) 
#df_errday.columns

In [11]:
for i in list(range(1,33)) :
        df_errday.iloc[:,i+1] = df_errday.iloc[:,i+1] + df_errday.iloc[:,i]

In [12]:
df_result = pd.merge(df_errtype, df_errday, on='user_id', how='inner')
#df_result.head()

## quality_data

# 광률님 quality 코드

In [ ]:

quality = pd.read_csv(PATH+'train_quality_data.csv')
quality.info()
quality.fillna(0) #quality_5변수에 NA값이 있음



#quality의 문자열변수,정수형변수 -> 실수형으로 바꿔주기
cols = ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
        'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
quality[cols] = quality[cols].replace({',':''}, regex= True) 
quality.iloc[:,3:16] = quality.iloc[:,3:16].astype(float) #quality_n 변수들 실수형으로 바꿔주기

#quality5만들기(quality의 일부 + -1의 개수 뽑아냄)

#minusQ 리스트는 각 user_id별 -1의 빈도수가 얼마나 나오는지 알기위해서 만드는 리스트임
#현재는 모든 quality 로그가 기록될때 -1인 퀄리티가 발생하면 minuQ리스트에 -1을 아니라면 0을 할당하는 식으로 새로운 열을 만듦
minusQ = []
for i in range(0,len(quality)): #데이터를 보면 quality 0,1,2,5,6,11 번째에서 동시에 -1이 발발하는것을 알 수 있었음.
  if (quality.iloc[i,3] == -1 or quality.iloc[i,4] == -1 or quality.iloc[i,5] == -1 or quality.iloc[i,8] == -1 or quality.iloc[i,9] == -1 or quality.iloc[i,14] == -1) :
  #if (quality.iloc[i,3] == -1 and quality.iloc[i,4] == -1 and quality.iloc[i,5] == -1 and quality.iloc[i,8] == -1 and quality.iloc[i,9] == -1 and quality.iloc[i,14] == -1) :
    minusQ.append(-1)
  else:
    minusQ.append(0)

quality['minusQ'] = minusQ #minusQ 열 추가해주기



#규칙을 찾아보면 quality는 무언가 누적합의 경우가 많음 그래서 가장 큰 숫자(max)를 대표값으로 지정하도록 코딩하였음
#minusQ의 경우는 -1의 발생 빈도를 볼 것이기 때문에, 예외적으로 count를 볼 것이기 떄문에 합(sum)으로 코딩하였음

def func(x):
    d = {}
    d['quality_0'] = x['quality_0'].max()
    d['quality_1'] = x['quality_1'].max()
    d['quality_2'] = x['quality_2'].max()
    d['quality_3'] = x['quality_3'].max()
    d['quality_4'] = x['quality_4'].max()
    d['quality_5'] = x['quality_5'].max()
    d['quality_6'] = x['quality_6'].max()
    d['quality_7'] = x['quality_7'].max()
    d['quality_8'] = x['quality_8'].max()
    d['quality_9'] = x['quality_9'].max()
    d['quality_10'] = x['quality_10'].max()
    d['quality_11'] = x['quality_11'].max()
    d['quality_12'] = x['quality_12'].max()
    d['minusQ'] = x['minusQ'].sum()
    return pd.Series(d, index= ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
                                'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12', 'minusQ'])

quality5 = quality.groupby(['user_id','time']).apply(func)
quality5


quality5 = quality5.groupby(['user_id']).sum()
quality5 = quality5.drop(columns = ['quality_3', 'quality_4']) 
#아래의 특징에 근거하여 일부 quality변수제거(이 외에도 다양한 경우로 조합이 가능할듯)
#quality 0,1,2,5,6,11 은 함께 -1을 가지며 발현되는 경우가 매우 많음
#quality 0와 2이 매우 비슷함
#quality 1과 11이 매우 비슷함(0과 2만큼은 아님)
#quality 3과 4는 모두 0
#quality 6의 누적합은 7
#quality 5의 누적합은 10
#quality 12는 대체로 0이며 큰 숫자가 없음
#quality 8은 아예 모르겠음

quality5

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        828624 non-null  int64  
 1   user_id     828624 non-null  int64  
 2   fwver       788544 non-null  object 
 3   quality_0   684192 non-null  float64
 4   quality_1   828624 non-null  int64  
 5   quality_2   788511 non-null  float64
 6   quality_3   828624 non-null  int64  
 7   quality_4   828624 non-null  int64  
 8   quality_5   828604 non-null  object 
 9   quality_6   828624 non-null  int64  
 10  quality_7   828624 non-null  object 
 11  quality_8   828624 non-null  object 
 12  quality_9   828624 non-null  object 
 13  quality_10  828624 non-null  object 
 14  quality_11  828624 non-null  int64  
 15  quality_12  828624 non-null  int64  
dtypes: float64(2), int64(8), object(6)
memory usage: 101.2+ MB


In [ ]:
df_result = pd.merge(df_result, quality5, left_on='user_id', right_on='user_id', how='left')
df_result.fillna(0, inplace = True)

print(df_result.shape)
display(df_result.head())

# one-hot encoding

In [39]:
train_err.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 13 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   int64 
 5   errcode   object
 6   year      int64 
 7   month     int64 
 8   day       int64 
 9   hour      int64 
 10  min       int64 
 11  sec       int64 
 12  date      int64 
dtypes: int64(10), object(3)
memory usage: 1.6+ GB


In [42]:
train_err['errtype_code'] = train_err['errtype'].astype(str)+'_'+train_err['errcode']

In [43]:
train_err.head(1)

,user_id,time,model_nm,fwver,errtype,errcode,year,month,day,hour,min,sec,date,errtype_code
0,10000,20201101025616,model_3,05.15.2138,15,1,2020,11,1,2,56,16,1101,15_1


In [45]:
cols1 = ['model_nm','fwver','errcode']
df_dummy1 = pd.get_dummies(train_err, columns=cols1,sparse=True, axis=1)

MemoryError: Unable to allocate 43.2 GiB for an array with shape (16554663, 2805) and data type uint8

In [ ]:
#cols2 = ['model_nm','fwver','errcode','errtype_code']
#df_dummy2 = pd.get_dummies(train_err, columns=cols2)

In [ ]:
df_dummy1.head(1)